In [ ]:
# ! pip install --upgrade --quiet transformers datasets pytorch-lightning sentencepiece

In [ ]:
!ls ../input/towardsnlp-train/bert-multi-cased-finetuned-xquadv1/checkpoint-*/
## 3836

In [ ]:
!ls ../input/epoch1-alonalbalakxlmrobertalargexquad

In [ ]:
## Import relevant packages

import os
import pandas as pd
import numpy as np

from datasets import Dataset as Dataset_pandas

import torch

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import RandomSampler

import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

from transformers import AutoModel, AutoModelForQuestionAnswering
from transformers import get_scheduler, AdamW
from transformers import AutoTokenizer

from pytorch_lightning import LightningDataModule, LightningModule, Trainer

from argparse import ArgumentParser
from tqdm import tqdm
import time, datetime

import random

In [ ]:
import collections
from functools import partial


In [ ]:
torch.cuda.empty_cache()

In [ ]:
def seed_everything(seed=772):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

In [ ]:
def parse_args():
	parser = ArgumentParser(description='Word Meaning Comparison')

	parser.add_argument('--data_path', '-d', type=str, default='../input/chaii-hindi-and-tamil-question-answering/test.csv')
	parser.add_argument('--model', '-m', type=str, default='../input/epoch2-alonalbalakxlmrobertalargexquad')

	parser.add_argument('--gpu', type=int, default=1)

	parser.add_argument('--max_len', type=int, default=384)
	parser.add_argument('--doc_stride', type=int, default=128)
	parser.add_argument('--max_answer_length', type=int, default=30)
	parser.add_argument('--best_answer_length', type=int, default=20)

	return parser.parse_known_args()[0]


args = parse_args()
print(args)

In [ ]:
device = torch.device('cuda' if args.gpu and torch.cuda.is_available() else 'cpu')

In [ ]:
device

## Utils

In [ ]:
# ref: https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb
def prepare_validation_features(examples, tokenizer, pad_on_right, max_length, doc_stride):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
# ref: https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb

def postprocess_qa_predictions(examples, tokenizer, features, raw_predictions, n_best_size=20, max_answer_length=30, squad_v2=False):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None  # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char:end_char],
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

## Data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(args.model)

In [ ]:
df = pd.read_csv(args.data_path)

data = Dataset_pandas.from_pandas(df)

print(data.column_names)


# ref: https://miguendes.me/how-to-pass-multiple-arguments-to-a-map-function-in-python
features = data.map(
    partial(
        prepare_validation_features, 
        tokenizer=tokenizer,
        pad_on_right=(tokenizer.padding_side == "right"), 
        max_length=args.max_len,
        doc_stride=args.doc_stride
    ),
    batched=True,
    remove_columns=data.column_names,
)

'''
features = data.map(
    lambda x: prepare_validation_features(x, 
        tokenizer=tokenizer,
        pad_on_right=(tokenizer.padding_side == "right"), 
        max_length=args.max_len,
        doc_stride=args.doc_stride
    ),
    batched=True,
    remove_columns=data.column_names,
)
'''

In [ ]:
## TODO
class QADataset(Dataset):
  def __init__(self, data):
    self.proc_data = data
    
  def __len__(self):
    return len(self.proc_data)
  
  def __getitem__(self, index):
    tokenized = self.proc_data[index]
    if 'token_type_ids' in tokenized:
        return (
                torch.tensor(tokenized["input_ids"], dtype=torch.long).to(device),
                torch.tensor(tokenized["attention_mask"], dtype=torch.long).to(device),
                torch.tensor(tokenized["token_type_ids"], dtype=torch.long).to(device)
                )
    else:
        return (
                torch.tensor(tokenized["input_ids"], dtype=torch.long).to(device),
                torch.tensor(tokenized["attention_mask"], dtype=torch.long).to(device)
                )
    
    # return (input_ids, attention_mask, token_type_ids)

In [ ]:

seed_everything()


test_data = QADataset(features)

test_dloader = DataLoader(test_data, 
                           batch_size=8, #len(test_data), #args.batch_size,
                           shuffle=False, # important!
                           )

In [ ]:
len(test_data), len(test_dloader)

In [ ]:
'''
for t in test_dloader:
  print(t[0].size())
  for el in t[0]:
    print(tokenizer.decode(el))
  break
'''

# Model

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(args.model)


In [ ]:
model.to(device)

In [ ]:
model.eval()

start_logits = []
end_logits = []

for _, batch in enumerate(test_dloader):
    print("+", end="")
    out = None
    if len(batch) == 2:
        (input_ids, attention_mask) = batch
        out = model(input_ids=input_ids,
                       attention_mask=attention_mask)
    else:
        (input_ids, attention_mask, token_type_ids) = batch
        # print(input_ids,input_ids.size())
        # print(attention_mask,attention_mask.size())
        # print(token_type_ids,token_type_ids.size())
        out = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)
        # print(out.start_logits, out.start_logits.size())
        # print(out.end_logits, out.start_logits.size())
        # break
        
    start_logits.append(out.start_logits.detach().cpu().numpy().copy())
    end_logits.append(out.end_logits.detach().cpu().numpy().copy())

In [ ]:
len(start_logits)

In [ ]:
start_logits = np.vstack(start_logits)
end_logits = np.vstack(end_logits)

In [ ]:
len(start_logits)

In [ ]:
pred = postprocess_qa_predictions(data, tokenizer, features, (start_logits, end_logits))

In [ ]:
submission = []
for p1, p2 in pred.items():
    submission.append((p1, p2.strip()))

In [ ]:
sample = pd.DataFrame(submission, columns=["id", "PredictionString"])
sample.to_csv("submission.csv", index=False)

In [ ]:
sample